### Please install the required Python modules/SDKs

In [1]:
! activate ai-azure-c1

import sys

sys.path.append("/opt/conda/envs/ai-azure-c1/lib/python3.8/site-packages")

## Step 4: Lighter Detection Using Azure Custom Vision

In [2]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os, time, uuid, glob

In [3]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

### Step 4.1. Creating Training & Prediting Client with credentials

In [4]:
CUSTOM_VISION_TRAINING_ENDPOINT_KEY = "29d644563b524d7c8dbff1f6695f8ed8"
CUSTOM_VISION_TRAINING_ENDPOINT = "https://anhthcustomvisions.cognitiveservices.azure.com/"
CUSTOM_VISION_TRAINING_ENDPOINT_RESOUCRE_ID = '/subscriptions/4c2045fc-9062-469b-a04d-fce56e0512a0/resourceGroups/aind-201553/providers/Microsoft.CognitiveServices/accounts/anhthcustomvisions'

In [5]:
CUSTOM_VISION_PREDICTION_KEY = "a35ab94feccb4685ab3ef2961da54b0b"
CUSTOM_VISION_PREDICTION_ENDPOINT = 'https://anhthcustomvisions-prediction.cognitiveservices.azure.com/'
CUSTOM_VISION_PREDICTION_ENDPOINT_RESOURCE_ID = "/subscriptions/4c2045fc-9062-469b-a04d-fce56e0512a0/resourceGroups/aind-201553/providers/Microsoft.CognitiveServices/accounts/anhthcustomvisions-Prediction"

In [6]:
train = ApiKeyCredentials(in_headers={"Training-key": CUSTOM_VISION_TRAINING_ENDPOINT_KEY})
train = CustomVisionTrainingClient(CUSTOM_VISION_TRAINING_ENDPOINT, train)
train.api_version

'3.4-preview'

In [7]:
pre = ApiKeyCredentials(in_headers={"Prediction-key": CUSTOM_VISION_PREDICTION_KEY})
pre = CustomVisionPredictionClient(CUSTOM_VISION_PREDICTION_ENDPOINT, pre)
pre.api_version

'3.1'

### Step 4.2. Get project ID and iteration ID from portal

In [15]:
project_id = train.get_projects()[0].id # Assume this is the only project on portal
print("Project ID: ", id)

Project ID:  5469dd51-9f65-4cec-a46c-bd089ed148d6


In [19]:
iteration_list = train.get_iterations(project_id)
iteration_id = None
for iteration in iteration_list:
    if iteration.name == "Iteration 2":
        iteration_id = iteration.id

### Step 4.3. Publish api with selected training iteration

In [20]:
publish_name = "Lighter_Detection_Udacity"
train.publish_iteration(project_id = project_id, iteration_id = iteration_id, publish_name = publish_name, prediction_id = CUSTOM_VISION_PREDICTION_ENDPOINT_RESOURCE_ID)
print ("Done!")

Done!


### Step 4.4. Perform prediction on test baggage images

In [21]:
def perform_prediction(predictor, publish_name, test_pathr="./lighter_detection_data_test/", threshold=0.8):
    list_img_path = glob.glob(os.path.join(test_pathr, '*.jpg'))
    for img_path in list_img_path:
        print("Predict the image path :", img_path)
        with open(img_path, "rb") as img:
            results = predictor.detect_image(project_id, publish_name, img.read())
            # Display the results.
            for prediction in results.predictions:
                if prediction.probability >= threshold:
                    print("\t" + prediction.tag_name +
                          ": {0:.2f}%".format(prediction.probability * 100))

In [24]:
perform_prediction(pre, publish_name)

Predict the image path : ./lighter_detection_data_test/lighter_test_set_4of5.jpg
	lighter: 97.33%
	lighter: 85.62%
Predict the image path : ./lighter_detection_data_test/lighter_test_set_5of5.jpg
	lighter: 86.60%
Predict the image path : ./lighter_detection_data_test/lighter_test_set_2of5.jpg
	lighter: 97.17%
	lighter: 96.00%
	lighter: 91.10%
Predict the image path : ./lighter_detection_data_test/lighter_test_set_3of5.jpg
	lighter: 97.80%
Predict the image path : ./lighter_detection_data_test/lighter_test_set_1of5.jpg
	lighter: 96.33%
	lighter: 87.60%
